# !pip install -U openpyxl

In [1]:
import os
import re
import pandas as pd
import numpy as np
from datetime import date, datetime
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')
from glob import glob

In [2]:
dh_inicio = datetime.today()

### CARGA DOS ARQUIVOS DOS CICLOS

In [3]:
# Caminho da pasta dos arquivos
caminho_pasta = r"C:\Projetos\PPSI\BASE\CICLOS-1-2-3-4"

# Lista os arquivos da pasta
arquivos = os.listdir(caminho_pasta)

# Filtra apenas arquivos (ignora subpastas)
arquivos = [f for f in arquivos if os.path.isfile(os.path.join(caminho_pasta, f))]

# Cria um DataFrame com os nomes
df = pd.DataFrame(arquivos, columns=["nome_arquivo"])

# Salva em Excel
#df.to_excel(r"C:\Users\MGI\Documents\COMAT\REFERENCIAS\CICLOS-1-2-3\CICLO_1_2_3_arquivos.xlsx", index=False)

print("Lista de arquivos salva com sucesso!")


Lista de arquivos salva com sucesso!


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nome_arquivo  627 non-null    object
dtypes: object(1)
memory usage: 5.0+ KB


### INFOS_BI_0-Indicadores

In [5]:
import os
import pandas as pd
import re

aba_desejada = "INFOS_BI_0"

# Lista para armazenar os DataFrames
lista_df = []

for arquivo in arquivos:
    caminho_arquivo = os.path.join(caminho_pasta, arquivo)
    
    try:
        df = pd.read_excel(caminho_arquivo, sheet_name=aba_desejada)

        df = df.iloc[:3]   # apenas linhas de 0 a 3

       
      # Renomeia as colunas com base nas posições
        novos_nomes = {
            df.columns[0]: 'Tipo_Indicador',            
            df.columns[1]: 'Valor_Indicador',
            df.columns[2]: 'Nivel_Maturidade',  
        }
        df.rename(columns=novos_nomes, inplace=True)
       
        #df['arquivo_origem'] = os.path.basename(arquivo)
        df['Nome da Origem.2'] = arquivo
        df['Nome da Origem.1'] = df['Nome da Origem.2'].str.extract(r'-(.*?)-')
        df['Ciclo_Entrega'] = df['Nome da Origem.2'].str[:2]
 
        # Garantir que os valores são float (caso tenham vindo como string)
        df['Valor_Indicador_num'] = pd.to_numeric(df['Valor_Indicador'], errors='coerce')

        # Arredondar para duas casas decimais
        df['Valor_Indicador_num'] = df['Valor_Indicador_num'].round(2)        
              
        df["Nivel_Maturidade"] = pd.cut(
        df["Valor_Indicador_num"],
        bins=[-0.01, 0.29, 0.49, 0.69, 0.89, 1.0],
        labels=["1. Inicial", "2. Básico", "3. Intermediário", "4. Em Aprimoramento", "5. Aprimorado"]
        )

        # Seleciona as colunas desejadas
       
        colunas = [
            'Tipo_Indicador', 'Valor_Indicador', 'Nivel_Maturidade', 'Nome da Origem.1', 
            'Nome da Origem.2', 'Ciclo_Entrega'
        ]      
        df = df[colunas]
       
        lista_df.append(df)
        #print(f"✅ Leitura concluída com sucesso: {arquivo}")
    except Exception as e:
        print(f"❌ Erro ao ler {arquivo}: {e}")

# Consolida todos os DataFrames
df_infos_bi_0 = pd.concat(lista_df, ignore_index=True)

# Resumo da execução

total_arquivos_consolidados = df_infos_bi_0['Nome da Origem.2'].nunique()
total_arquivos = len(arquivos)
sucesso = len(lista_df)
erros = total_arquivos - sucesso

print("\n📊 Resumo da execução:")
print(f"🗂️ Total de arquivos considerados: {total_arquivos}")
print(f"✅ Arquivos lidos com sucesso: {sucesso}")
print(f"❌ Arquivos com erro: {erros}")
 
print(f"📦 Total de arquivos únicos no consolidado (Nome da Origem.2): {total_arquivos_consolidados}")

print("✅ ✅ Consolidação concluída com sucesso!")

❌ Erro ao ler C1-UNILA-ferramenta_frameworkpsi_pt-2 (1).xlsm: Worksheet named 'INFOS_BI_0' not found

📊 Resumo da execução:
🗂️ Total de arquivos considerados: 627
✅ Arquivos lidos com sucesso: 626
❌ Arquivos com erro: 1
📦 Total de arquivos únicos no consolidado (Nome da Origem.2): 626
✅ ✅ Consolidação concluída com sucesso!


In [6]:
colunas1 = [
            'Tipo_Indicador', 'Valor_Indicador', 'Nivel_Maturidade', 'Nome da Origem.1', 
            'Nome da Origem.2', 'Ciclo_Entrega'
        ] 
        
df = df_infos_bi_0
for col in colunas1:
    if col in df.columns:
        vazios = df[col].isnull().sum() + (df[col].astype(str).str.strip() == '').sum()
        print(f"{col}: {vazios} vazios")
    else:
        print(f"Coluna '{col}' não encontrada")

Tipo_Indicador: 0 vazios
Valor_Indicador: 0 vazios
Nivel_Maturidade: 29 vazios
Nome da Origem.1: 0 vazios
Nome da Origem.2: 0 vazios
Ciclo_Entrega: 0 vazios


In [7]:
df_infos_bi_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Tipo_Indicador    1252 non-null   object  
 1   Valor_Indicador   1252 non-null   object  
 2   Nivel_Maturidade  1223 non-null   category
 3   Nome da Origem.1  1252 non-null   object  
 4   Nome da Origem.2  1252 non-null   object  
 5   Ciclo_Entrega     1252 non-null   object  
dtypes: category(1), object(5)
memory usage: 50.5+ KB


In [8]:
#df_consolidado['ID CONTROLE'] = df_consolidado['ID CONTROLE'].astype('object')

In [9]:
#df_consolidado['Valor_Indicador'] = pd.to_numeric(df_consolidado['Valor_Indicador'], errors='coerce').round(2)

In [10]:
# Função para extrair o número do ciclo (C1 → 1, C4 → 4)
#def extrair_numero_ciclo(ciclo):
#    match = re.search(r'C(\d+)', str(ciclo))
#    return int(match.group(1)) if match else None

# Aplica a extração no DataFrame consolidado
#df_consolidado['numero_ciclo'] = df_consolidado['maior_ciclo'].apply(extrair_numero_ciclo)

# Agrupa por órgão (ajuste o nome da coluna conforme seu caso real)
#df_maior_ciclo_por_orgao = df_consolidado.loc[#
#    df_consolidado.groupby('Nome da Origem.1')['numero_ciclo'].idxmax()
#][['Nome da Origem.1', 'maior_ciclo']].rename(columns={'maior_ciclo': 'maior_ciclo_por_orgao'})

# Junta de volta ao DataFrame consolidado, se quiser
#df_consolidado = df_consolidado.merge(df_maior_ciclo_por_orgao, on='Nome da Origem.1', how='left')


In [11]:
# Seleciona as colunas desejadas
#colunas = [
#    'Nome da Origem.1', 'Nome da Origem.2', 'Ciclo_Entrega', 
#    'ID CONTROLE', 'NOME CONTROLE', 'Valor_Indicador_Controle', 'Nivel_Maturidade_Controle'
#]
#df_consolidado = df_consolidado[colunas]

In [12]:
#df_consolidado['Valor_Indicador_Controle_Priv'] = df_consolidado['Valor_Indicador_Controle_Priv'].astype(float)

In [13]:
# Seleciona as colunas desejadas
#colunas = [
#    'Nome da Origem.1', 'Nome da Origem.2', 'Ciclo de Entrega', 'maior_ciclo_por_orgao', 
#    'ID CONTROLE', 'NOME CONTROLE', 'Valor_Indicador_Controle_Priv', 'Nivel_Maturidade_Controle_Priv'
#]
#df_consolidado = df_consolidado[colunas].rename(columns={
#    'maior_ciclo_por_orgao': 'Ciclo de Entrega2'
#})

In [14]:
#mapa_maturidade = {
#    'Inicial': '1. Inicial',
#    'Básico': '2. Básico',
#    'Intermediário': '3. Intermediário',
#    'Em Aprimoramento': '4. Em Aprimoramento',
#    'Aprimorardo': '5. Aprimorardo'   
#}

#df_consolidado['Nivel_Maturidade_Controle'] = df_consolidado['Nivel_Maturidade_Controle'].map(mapa_maturidade).fillna('')


In [15]:
#df_consolidado['Valor_Indicador_Controle'] = pd.to_numeric(df_consolidado['Valor_Indicador_Controle'], errors='coerce').round(2)

In [16]:
#df_consolidado['Mais_Recente'] = np.where(df_consolidado['Ciclo de Entrega'] == df_consolidado['Ciclo de Entrega2'], 'Sim', 'Não')

In [17]:
#df_consolidado['Mais_Recente_2'] = df_consolidado['Ciclo de Entrega'].astype(str) + df_consolidado['Ciclo de Entrega2'].astype(str)

In [18]:
#df_consolidado.info()

In [19]:
# Salva em Excel
#df_consolidado.to_excel(r"C:\Projetos\PPSI\Doc_Apoio_BI\INFOS_BI_2-Controles-Priv_C.xlsx", sheet_name = "INFOS_BI_2-Controles-Priv_C", index=False)

In [20]:
#df_consolidado['Valor_Indicador_Controle'] = df_consolidado['Valor_Indicador_Controle'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')

In [21]:
#df_consolidado['Valor_Indicador_Controle'] = pd.to_numeric(df_consolidado['Valor_Indicador_Controle'], errors='coerce').round(2)

In [22]:
df_consolidado = df_infos_bi_0

In [23]:
df_consolidado['Ciclo_Entrega'] = df_consolidado['Ciclo_Entrega'].astype(str).str.strip().str.upper()


In [24]:
# Filtrar somente ciclos válidos que seguem o padrão "C" seguido de número
ciclos_ordenados = sorted(
    [c for c in df_consolidado['Ciclo_Entrega'].unique() if c.startswith('C') and c[1:].isdigit()],
    key=lambda x: int(x[1:])
)


In [25]:
df_resultado = pd.DataFrame()
chave = ['Nome da Origem.1', 'Tipo_Indicador']

for i, ciclo in enumerate(ciclos_ordenados):
    ciclos_ate_agora = ciclos_ordenados[:i+1]

    df_filtrado = df_consolidado[df_consolidado['Ciclo_Entrega'].isin(ciclos_ate_agora)].copy()

    # Mapeia a ordem dos ciclos dinamicamente
    ordem_map = {c: idx for idx, c in enumerate(ciclos_ate_agora)}
    df_filtrado['ordem_ciclo'] = df_filtrado['Ciclo_Entrega'].map(ordem_map)

    # Ordena para pegar o mais recente por grupo
    df_filtrado = (
        df_filtrado
        .sort_values(chave + ['ordem_ciclo'], ascending=[True]*len(chave) + [False])
        .drop_duplicates(subset=chave, keep='first')
    )

    df_filtrado['Ciclo_Contexto'] = ciclo
    df_resultado = pd.concat([df_resultado, df_filtrado.drop(columns='ordem_ciclo')], ignore_index=True)


In [26]:
#colunas_para_remover = ['Mais_Recente', 'Mais_Recente_2', 'Ciclo de Entrega2']
#df_resultado = df_resultado.drop(columns=colunas_para_remover, errors='ignore')


In [27]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Tipo_Indicador    1400 non-null   object  
 1   Valor_Indicador   1400 non-null   object  
 2   Nivel_Maturidade  1369 non-null   category
 3   Nome da Origem.1  1400 non-null   object  
 4   Nome da Origem.2  1400 non-null   object  
 5   Ciclo_Entrega     1400 non-null   object  
 6   Ciclo_Contexto    1400 non-null   object  
dtypes: category(1), object(6)
memory usage: 67.3+ KB


In [28]:
# Salva em Excel
#df_resultado.to_excel(r"C:\Projetos\PPSI\Doc_Apoio_BI\INFOS_BI_2-Controles-Priv_Resultado_C.xlsx", sheet_name = "INFOS_BI_2-Controles-Priv_Resultado_C", index=False)

In [29]:

#df_resultado['Valor_Indicador_Controle_Priv'] = df_resultado['Valor_Indicador_Controle_Priv'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')


In [30]:
#df_resultado['Valor_Indicador_Controle_Priv'] = pd.to_numeric(df_resultado['Valor_Indicador_Controle_Priv'], errors='coerce').round(2)

In [31]:
# Salva em Excel
#df_resultado.to_csv(r"C:\Projetos\PPSI\Doc_Apoio_BI\INFOS_BI_2-Controles-Priv_Resultado_C.csv", index=False)

In [32]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Tipo_Indicador    1400 non-null   object  
 1   Valor_Indicador   1400 non-null   object  
 2   Nivel_Maturidade  1369 non-null   category
 3   Nome da Origem.1  1400 non-null   object  
 4   Nome da Origem.2  1400 non-null   object  
 5   Ciclo_Entrega     1400 non-null   object  
 6   Ciclo_Contexto    1400 non-null   object  
dtypes: category(1), object(6)
memory usage: 67.3+ KB


In [33]:
# Manter a coluna como float
df_resultado['Valor_Indicador'] = pd.to_numeric(df_resultado['Valor_Indicador'], errors='coerce')

# Exibir com vírgula (sem alterar tipo para 'object')
df_resultado['Valor_Indicador'] = df_resultado['Valor_Indicador'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')

# Mostrar o DataFrame
#print(df_resultado[['Valor_Indicador_Controle_Priv', 'Valor_Indicador_Controle_Priv_Formatado']])


In [34]:
# 1. Converte colunas numéricas com vírgula decimal
#for col in df_resultado.select_dtypes(include=['float', 'int']).columns:
#    df_resultado[col] = df_resultado[col].map(lambda x: f"{x:,.2f}".replace('.', ',') if pd.notnull(x) else x)

In [35]:
#df_resultado['Valor_Indicador_Controle_Priv'] = pd.to_numeric(df_resultado['Valor_Indicador_Controle_Priv'], errors='coerce').round(2)

In [36]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_infos_bi_0['Ciclo_Entrega'] = df_infos_bi_0['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [37]:
# Salva em Excel
df_infos_bi_0.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\INFOS_BI_0-Indicadores_C.xlsx", sheet_name = "INFOS_BI_0-Indicadores_C", index=False)

In [38]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_resultado['Ciclo_Entrega'] = df_resultado['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')
df_resultado['Ciclo_Contexto'] = df_resultado['Ciclo_Contexto'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [39]:
# Salva em Excel
df_resultado.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\INFOS_BI_0-Indicadores_Recente.xlsx", sheet_name = "INFOS_BI_0-Indicadores_Recente", index=False)

In [40]:
# Salvando o DataFrame em CSV com o separador correto e garantindo que o ID seja tratado como texto
#df_resultado.to_csv(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\INFOS_BI_0-Indicadores_Recente.csv",
#    sep=';',  # Usando ponto e vírgula como separador (dependendo da configuração regional)
#    index=False,  # Não incluir o índice
#    encoding='utf-8-sig',  # Codificação para garantir compatibilidade com o Excel
#    float_format="%.2f",  # Formato de números com 2 casas decimais
#    quotechar='"',  # Envolver os campos em aspas para garantir que os valores não sejam alterados
#    quoting=1  # Forçar o uso de aspas
#)

In [41]:
df_resultado.head()

,Tipo_Indicador,Valor_Indicador,Nivel_Maturidade,Nome da Origem.1,Nome da Origem.2,Ciclo_Entrega,Ciclo_Contexto
0,IPRIV,"0,52",3. Intermediário,ABIN,C1-ABIN-Ferramenta_frameworkpsi_pt-3-basica-se...,Ciclo 1,Ciclo 1
1,ISEG,"0,73",4. Em Aprimoramento,ABIN,C1-ABIN-Ferramenta_frameworkpsi_pt-3-basica-se...,Ciclo 1,Ciclo 1
2,IPRIV,"0,41",2. Básico,AEB,C1-AEB-PPSI - Ciclo 1 - Framework.xlsx,Ciclo 1,Ciclo 1
3,ISEG,"0,73",4. Em Aprimoramento,AEB,C1-AEB-PPSI - Ciclo 1 - Framework.xlsx,Ciclo 1,Ciclo 1
4,IPRIV,"0,23",1. Inicial,AGU,C1-AGU-Cópia de PPSI-AGU-Framework_Ciclo_1-em ...,Ciclo 1,Ciclo 1


In [42]:
dh_termino = datetime.today()
duracao = dh_termino - dh_inicio
print(dh_inicio)
print(dh_termino)
print(duracao)

2025-05-27 09:26:37.034554
2025-05-27 09:29:31.375239
0:02:54.340685


In [43]:
colunas1 = [
            'Tipo_Indicador', 'Valor_Indicador', 'Nivel_Maturidade', 'Nome da Origem.1', 
            'Nome da Origem.2', 'Ciclo_Entrega'
        ] 
df = df_resultado
for col in colunas1:
    if col in df.columns:
        vazios = df[col].isnull().sum() + (df[col].astype(str).str.strip() == '').sum()
        print(f"{col}: {vazios} vazios")
    else:
        print(f"Coluna '{col}' não encontrada")

Tipo_Indicador: 0 vazios
Valor_Indicador: 31 vazios
Nivel_Maturidade: 31 vazios
Nome da Origem.1: 0 vazios
Nome da Origem.2: 0 vazios
Ciclo_Entrega: 0 vazios
